## Vstavané reprezentácie

V našom predchádzajúcom príklade sme pracovali s vysokodimenzionálnymi vektormi bag-of-words s dĺžkou `vocab_size` a explicitne sme prevádzali nízkodimenzionálne pozičné reprezentácie na riedke one-hot reprezentácie. Táto one-hot reprezentácia nie je pamäťovo efektívna, navyše, každé slovo je spracované nezávisle od ostatných, t.j. one-hot zakódované vektory nevyjadrujú žiadnu sémantickú podobnosť medzi slovami.

V tejto časti budeme pokračovať v skúmaní datasetu **News AG**. Na začiatok načítajme dáta a získajme niektoré definície z predchádzajúceho notebooku.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Čo je embedding?

Myšlienka **embeddingu** spočíva v reprezentácii slov pomocou nižšej dimenzie hustých vektorov, ktoré určitým spôsobom odrážajú sémantický význam slova. Neskôr si povieme, ako vytvoriť zmysluplné word embeddings, ale zatiaľ si predstavme embeddingy ako spôsob zníženia dimenzionality vektora slova.

Embedding vrstva teda prijme slovo ako vstup a vygeneruje výstupný vektor s určenou veľkosťou `embedding_size`. V istom zmysle je veľmi podobná vrstve `Linear`, ale namiesto prijímania vektora zakódovaného v one-hot formáte dokáže prijať číslo slova ako vstup.

Použitím embedding vrstvy ako prvej vrstvy v našej sieti môžeme prejsť od modelu bag-of-words k modelu **embedding bag**, kde najskôr každé slovo v našom texte prevedieme na zodpovedajúci embedding a potom vypočítame nejakú agregačnú funkciu nad všetkými týmito embeddingami, ako napríklad `sum`, `average` alebo `max`.

![Obrázok zobrazujúci embedding klasifikátor pre päť slov v sekvencii.](../../../../../translated_images/sk/embedding-classifier-example.b77f021a7ee67eee.webp)

Naša klasifikačná neurónová sieť začne embedding vrstvou, potom agregačnou vrstvou a na vrchu bude lineárny klasifikátor:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Práca s rôznou veľkosťou sekvencie premenných

V dôsledku tejto architektúry je potrebné vytvárať minibatchy pre našu sieť určitým spôsobom. V predchádzajúcej jednotke, pri použití metódy bag-of-words, mali všetky BoW tenzory v minibatchi rovnakú veľkosť `vocab_size`, bez ohľadu na skutočnú dĺžku textovej sekvencie. Keď prejdeme na slovné vektory (word embeddings), skončíme s rôznym počtom slov v každom textovom vzorku, a pri kombinovaní týchto vzoriek do minibatchov bude potrebné použiť nejaké doplnenie (padding).

Toto je možné dosiahnuť použitím rovnakej techniky, ktorá zahŕňa poskytnutie funkcie `collate_fn` pre zdroj dát:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Tréning klasifikátora embeddingov

Teraz, keď sme definovali správny dataloader, môžeme model trénovať pomocou tréningovej funkcie, ktorú sme definovali v predchádzajúcej jednotke:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Poznámka**: Trénujeme tu iba na 25k záznamoch (menej ako jeden celý epoch) kvôli úspore času, ale môžete pokračovať v tréningu, napísať funkciu na tréning počas viacerých epoch a experimentovať s parametrom rýchlosti učenia, aby ste dosiahli vyššiu presnosť. Mali by ste byť schopní dosiahnuť presnosť približne 90%.


### Vrstva EmbeddingBag a reprezentácia sekvencií s premenlivou dĺžkou

V predchádzajúcej architektúre sme museli všetky sekvencie doplniť na rovnakú dĺžku, aby sa zmestili do minibatchu. Toto nie je najefektívnejší spôsob reprezentácie sekvencií s premenlivou dĺžkou - iný prístup by bol použiť **offset** vektor, ktorý by obsahoval posuny všetkých sekvencií uložených v jednom veľkom vektore.

![Obrázok zobrazujúci reprezentáciu sekvencií pomocou offset vektora](../../../../../translated_images/sk/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: Na obrázku vyššie je zobrazená sekvencia znakov, ale v našom príklade pracujeme so sekvenciami slov. Avšak, základný princíp reprezentácie sekvencií pomocou offset vektora zostáva rovnaký.

Na prácu s offset reprezentáciou používame vrstvu [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Je podobná vrstve `Embedding`, ale ako vstup berie obsahový vektor a offset vektor, a navyše obsahuje vrstvu na priemerovanie, ktorá môže byť `mean`, `sum` alebo `max`.

Tu je upravená sieť, ktorá používa `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Na prípravu dátovej sady na trénovanie musíme poskytnúť konverznú funkciu, ktorá pripraví vektor posunu:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Všimnite si, že na rozdiel od všetkých predchádzajúcich príkladov naša sieť teraz prijíma dva parametre: dátový vektor a offsetový vektor, ktoré majú rôzne veľkosti. Podobne aj náš načítač dát nám poskytuje 3 hodnoty namiesto 2: ako vlastnosti sú poskytované textové aj offsetové vektory. Preto musíme mierne upraviť našu tréningovú funkciu, aby sme to zohľadnili:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Sémantické vektory: Word2Vec

V našom predchádzajúcom príklade sa vrstva modelu na vkladanie naučila mapovať slová na vektorové reprezentácie, avšak táto reprezentácia nemala veľký sémantický význam. Bolo by dobré naučiť sa takú vektorovú reprezentáciu, kde by podobné slová alebo synonymá zodpovedali vektorom, ktoré sú si blízke podľa nejakej vektorovej vzdialenosti (napr. euklidovskej vzdialenosti).

Aby sme to dosiahli, musíme náš model na vkladanie predtrénovať na veľkej zbierke textov špecifickým spôsobom. Jedným z prvých spôsobov, ako trénovať sémantické vektory, je metóda nazývaná [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Táto metóda je založená na dvoch hlavných architektúrach, ktoré sa používajú na vytvorenie distribuovanej reprezentácie slov:

 - **Kontinuálny model bag-of-words** (CBoW) — v tejto architektúre trénujeme model na predpovedanie slova na základe okolitých slov v kontexte. Ak máme ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, cieľom modelu je predpovedať $W_0$ na základe $(W_{-2},W_{-1},W_1,W_2)$.
 - **Kontinuálny skip-gram** je opakom CBoW. Model používa okno okolitých slov v kontexte na predpovedanie aktuálneho slova.

CBoW je rýchlejší, zatiaľ čo skip-gram je pomalší, ale lepšie reprezentuje zriedkavé slová.

![Obrázok zobrazujúci algoritmy CBoW a Skip-Gram na konverziu slov na vektory.](../../../../../translated_images/sk/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Na experimentovanie s Word2Vec vektormi predtrénovanými na Google News dataset môžeme použiť knižnicu **gensim**. Nižšie nájdeme slová najviac podobné slovu 'neural'.

> **Note:** Keď prvýkrát vytvárate vektorové reprezentácie slov, ich sťahovanie môže chvíľu trvať!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Môžeme tiež vypočítať vektorové zakotvenia zo slova, ktoré sa použijú pri trénovaní klasifikačného modelu (z dôvodu prehľadnosti zobrazujeme iba prvých 20 komponentov vektora):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Skvelá vec na semantických vkladoch je, že môžete manipulovať s vektorovým kódovaním na zmenu semantiky. Napríklad môžeme požiadať o nájdenie slova, ktorého vektorová reprezentácia by bola čo najbližšie k slovám *kráľ* a *žena*, a čo najďalej od slova *muž*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Obe metódy, CBoW a Skip-Grams, sú „prediktívne“ vektory, pretože berú do úvahy iba lokálne kontexty. Word2Vec nevyužíva globálny kontext.

**FastText** rozširuje Word2Vec tým, že sa učí vektorové reprezentácie pre každé slovo a znakové n-gramy nachádzajúce sa v rámci každého slova. Hodnoty týchto reprezentácií sa potom pri každom kroku tréningu spriemerujú do jedného vektora. Aj keď to pridáva veľa dodatočných výpočtov počas pred-tréningu, umožňuje to, aby vektorové reprezentácie slov obsahovali informácie o podslovách.

Ďalšia metóda, **GloVe**, využíva myšlienku matice spoluvýskytu a používa neurónové metódy na rozklad matice spoluvýskytu do výraznejších a nelineárnych vektorov slov.

Môžete si vyskúšať príklad zmenou vektorových reprezentácií na FastText a GloVe, keďže gensim podporuje niekoľko rôznych modelov vektorových reprezentácií slov.


## Použitie predtrénovaných vektorov v PyTorch

Môžeme upraviť vyššie uvedený príklad tak, aby sme predvyplnili maticu v našej vrstve embedding pomocou sémantických vektorov, ako je Word2Vec. Musíme brať do úvahy, že slovníky predtrénovaných vektorov a nášho textového korpusu sa pravdepodobne nebudú zhodovať, takže inicializujeme váhy pre chýbajúce slová náhodnými hodnotami:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Teraz poďme trénovať náš model. Všimnite si, že čas potrebný na trénovanie modelu je výrazne dlhší ako v predchádzajúcom príklade, kvôli väčšej veľkosti vrstvy embedding a teda oveľa vyššiemu počtu parametrov. Tiež, kvôli tomu môžeme potrebovať trénovať náš model na viacerých príkladoch, ak chceme vyhnúť sa overfittingu.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

V našom prípade nepozorujeme výrazné zvýšenie presnosti, čo pravdepodobne súvisí s odlišnými slovníkmi.  
Na prekonanie problému s rôznymi slovníkmi môžeme použiť jedno z nasledujúcich riešení:  
* Znovu natrénovať model word2vec na našom slovníku  
* Načítať náš dataset so slovníkom z predtrénovaného modelu word2vec. Slovník použitý na načítanie datasetu je možné špecifikovať počas načítania.  

Druhý prístup sa zdá jednoduchší, najmä preto, že PyTorch `torchtext` framework obsahuje vstavanú podporu pre embeddingy. Môžeme napríklad vytvoriť slovník založený na GloVe nasledujúcim spôsobom:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Načítaná slovná zásoba má nasledujúce základné operácie:
* Slovník `vocab.stoi` nám umožňuje previesť slovo na jeho index v slovníku
* `vocab.itos` robí opak - prevádza číslo na slovo
* `vocab.vectors` je pole vektorov zapuzdrenia, takže na získanie zapuzdrenia slova `s` musíme použiť `vocab.vectors[vocab.stoi[s]]`

Tu je príklad manipulácie s vektormi na demonštráciu rovnice **kind-man+woman = queen** (musel som trochu upraviť koeficient, aby to fungovalo):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Na natrénovanie klasifikátora pomocou týchto embeddingov najprv musíme zakódovať náš dataset pomocou slovníka GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Ako sme videli vyššie, všetky vektorové vloženia sú uložené v matici `vocab.vectors`. To umožňuje veľmi jednoduché načítanie týchto váh do váh vrstvy vloženia pomocou jednoduchého kopírovania:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Jedným z dôvodov, prečo nevidíme významné zvýšenie presnosti, je skutočnosť, že niektoré slová z nášho datasetu chýbajú v predtrénovanej slovnej zásobe GloVe, a preto sú v podstate ignorované. Aby sme túto skutočnosť prekonali, môžeme trénovať vlastné vektorové reprezentácie na našom datasete.


## Kontextové vkladania

Jedným z hlavných obmedzení tradičných predtrénovaných reprezentácií vkladania, ako je Word2Vec, je problém rozlíšenia významu slov. Hoci predtrénované vkladania dokážu zachytiť určitý význam slov v kontexte, každý možný význam slova je zakódovaný do toho istého vkladania. To môže spôsobovať problémy v následných modeloch, pretože mnohé slová, ako napríklad slovo 'hrať', majú rôzne významy v závislosti od kontextu, v ktorom sú použité.

Napríklad slovo 'hrať' v týchto dvoch vetách má úplne odlišný význam:
- Išiel som na **hru** do divadla.
- Ján chce **hrať** so svojimi priateľmi.

Predtrénované vkladania vyššie reprezentujú oba tieto významy slova 'hrať' v tom istom vkladaní. Aby sme prekonali toto obmedzenie, musíme vytvoriť vkladania založené na **jazykovom modeli**, ktorý je trénovaný na veľkom korpuse textu a *vie*, ako môžu byť slová poskladané v rôznych kontextoch. Diskusia o kontextových vkladaní je mimo rozsahu tohto tutoriálu, ale vrátime sa k nim pri rozprávaní o jazykových modeloch v ďalšej jednotke.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
